In [23]:
from azure.ai.ml import command
from azure.ai.ml.entities import Data
from azure.ai.ml import Input, Output, load_component
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import MLClient
from azure.ai.ml.dsl import pipeline
# Identity
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
# Schedule
from azure.ai.ml.constants import TimeZone
from azure.ai.ml.entities import (
    JobSchedule,
    CronTrigger,
    RecurrenceTrigger,
    RecurrencePattern,
)
from datetime import datetime

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "gpu-cluster-low"
print(ml_client.compute.get(cluster_name))

Found the config file in: /config.json


enable_node_public_ip: true
id: /subscriptions/96c5f48d-9bf8-4453-a190-e2ce1e4fcea5/resourceGroups/rg-aml/providers/Microsoft.MachineLearningServices/workspaces/aml/computes/gpu-cluster-low
identity:
  principal_id: 076b32bf-1545-436f-b3ad-b665ecc2e68d
  tenant_id: e0793d39-0939-496d-b129-198edd916feb
  type: system_assigned
idle_time_before_scale_down: 120
location: eastus2
max_instances: 1
min_instances: 0
name: gpu-cluster-low
provisioning_state: Succeeded
size: STANDARD_NC4AS_T4_V3
ssh_public_access_enabled: false
tier: low_priority
type: amlcompute



In [24]:
parent_dir = "."
ccb_pipeline_single_step = load_component(source=parent_dir + "/ccb_pipeline.yml")

# Construct pipeline
@pipeline(name='ccb_pipeline',display_name='CryptoBotPipeline',experiment_name='ccb_v3')
def ccb_pipeline(
    data_path,
):
    """Single script pipeline to dowload data / train / score models"""
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    ccb_script = ccb_pipeline_single_step(
        data_path=data_path,
    )
    ccb_script.outputs.output_folder = Output(
        type=AssetTypes.URI_FOLDER,
        # path="azureml:ccb_data_folder:1",
        path="azureml://subscriptions/<your subscription id>/resourcegroups/<your rg id>/workspaces/<your aml id>/datastores/<your datastore id>/paths/data/"
    )

    # Return: pipeline outputs
    return {
        "data_folder": ccb_script.outputs.output_folder,
    }

In [25]:
# set run time settings
pipeline_job = ccb_pipeline(
    data_path=Input(type="uri_folder", path="azureml:ccb_data_folder:1"),
)

# set pipeline level compute
pipeline_job.settings.default_compute = "gpu-cluster-low"
# force tu run each time
pipeline_job.settings.force_rerun = True

# A. Submit to run __once__

In [20]:
# Submit pipeline job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="ccb_v3"
)
pipeline_job

Uploading v3-aml (7.83 MBs): 100%|██████████| 7830622/7830622 [00:00<00:00, 34040886.76it/s]




Experiment,Name,Type,Status,Details Page
ccb_v3,yellow_tree_5mk5d2d5lx,pipeline,Preparing,Link to Azure Machine Learning studio


# B. Schedule pipeline

In [26]:
schedule_name = "ccb_daily"

schedule_start_time = datetime.utcnow()
recurrence_trigger = RecurrenceTrigger(
    frequency="day",
    interval=1,
    schedule=RecurrencePattern(hours=5, minutes=[0]),
    start_time=schedule_start_time,
    time_zone=TimeZone.UTC,
)

job_schedule = JobSchedule(
    name=schedule_name,
    display_name='ccb daily', 
    trigger=recurrence_trigger,
    create_job=pipeline_job
)

In [27]:
job_schedule = ml_client.schedules.begin_create_or_update(
    schedule=job_schedule
).result()
print(job_schedule)

..name: ccb_daily
display_name: ccb daily
is_enabled: true
provisioning_state: Succeeded
creation_context:
  created_at: '2023-06-06T17:54:39.306307+00:00'
  created_by: Cinert
  created_by_type: User
  last_modified_at: '2023-06-14T10:10:51.991889+00:00'
  last_modified_by: Cinert
  last_modified_by_type: User
trigger:
  start_time: '2023-06-14 10:10:44.162800'
  time_zone: UTC
  type: recurrence
  frequency: day
  interval: 1
  schedule:
    hours:
    - 5
    minutes:
    - 0
create_job:
  description: Cina Crypto Bot single script pipeline to dowload data / train / score
    models
  display_name: CinaCryptoBotPipeline
  status: NotStarted
  experiment_name: ccb_v3
  type: pipeline
  settings:
    force_rerun: true
    default_compute: azureml:/subscriptions/96c5f48d-9bf8-4453-a190-e2ce1e4fcea5/resourceGroups/rg-aml/providers/Microsoft.MachineLearningServices/workspaces/aml/computes/gpu-cluster-low
  inputs:
    data_path:
      mode: ro_mount
      type: uri_folder
      path: azu